# Proses Analisis Sentimen Pada Ulasan Aplikasi Peduli Lindungi

### Menginstall *package* yang dibutuhkan

In [ ]:
!pip3 uninstall googletrans
!pip3 install googletrans==3.1.0a0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.9 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16352 sha256=22871f3376a3959ce968b6a445cb2ffd67347f445b7c21a11f228668fa59a46f
  Stored in directory: /root/.cache/pip/wheels/50/5d/3c/8477d0af4ca2b8b1308812c09f1930863caeebc762fe265a95
Successfully built googletrans


### Mengimpor *library* yang dibutuhkan

In [ ]:
from googletrans import Translator
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd
import re
import string

# Filtering
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Stemming
from nltk.stem.porter import PorterStemmer
from sklearn.pipeline import Pipeline

# Labeling
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

# TF-IDF
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Splitting data
from sklearn.model_selection import train_test_split

# Model
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, auc, roc_curve

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


###  Mengimpor dan menampilkan data dalam bentuk tabel

In [ ]:
path = 'Scrapped_data.csv'
data = pd.read_csv(path)
data.head()

FileNotFoundError: ignored

In [ ]:
data.head(5827)

### Menampilkan jumlah baris dan kolom data

In [ ]:
data.shape

### Menghilangkan variabel yang tidak dipakai

In [ ]:
df_data = data.copy()
df_data = data.drop(columns = ['Nama reviewer','Rating','Tanggal ulasan'])


### Menerjemahkan ke dalam bahasa Inggris

In [ ]:
translator = Translator()
df_data['Ulasan'] = df_data['Ulasan'].apply(translator.translate, src='id', dest='en').apply(getattr, args=('text',))

In [ ]:
df_data_en = df_data.copy()
df_data_en.to_csv('data_en.csv', index=False)
df_data_en.head()

### *Data Preprocessing*
#### *Case Folding*

In [ ]:
def case_folding(content):
  content = content.lower()                             # mengecilkan huruf
  content = content.strip(' ')                          # menghapus spasi diawal dan diakhir string
  content = re.sub(r'[?|$|.,|!_:)(-+)]', '', content)   # mengganti tanda ?|$ dsb dengan kosong
  content = re.sub(r'\d', '', content)                  # menghapus angka
  content = re.sub('[^\w\s]', '',content)               # menghapus punctuations
  content = re.sub('  ', ' ',content)                   # menghapus dua spasi
  return content
df_data_en['Ulasan'] = df_data_en['Ulasan'].apply(case_folding)
df_data_en.head()

In [ ]:
df_data_en.to_csv('data_CF.csv', index=False)

#### *Tokenizing*

In [ ]:
def token(content):
  nstr = content.split(' ')
  dat = []
  a = -1
  for hu in nstr:
    a = a+1
  if hu == '':
    dat.append(a)
  return nstr
df_data_en['Ulasan'] = df_data_en['Ulasan'].apply(token)
df_data_en.head()

In [ ]:
df_data_en.to_csv('data_Toke.csv', index=False)

#### *Filtering*

In [ ]:
def stopwords_removal(content):
  filtering = stopwords.words('english')
  x = []
  data = []
  def myFunc(x):
    if x in filtering:
      return False
    else:
      return True
  fit = filter(myFunc, content)
  for x in fit:
    data.append(x)
  return data
df_data_en['Ulasan'] = df_data_en['Ulasan'].apply(stopwords_removal)
df_data_en.head()

In [ ]:
df_data_en.to_csv('data_Filter.csv', index=False)

#### *Stemming*

In [ ]:
def stemming(content):
  stemmer = nltk.porter.PorterStemmer()
  do = []
  for w in content:
    dt = stemmer.stem(w)
    do.append(dt)
  d_clean = []
  d_clean = " ".join(do)
  return d_clean
df_data_en['Ulasan'] = df_data_en['Ulasan'].apply(stemming)

df_data_en.to_csv('data_cleans.csv', index=False)
data_cleans = pd.read_csv('data_cleans.csv', encoding='latin1')
data_cleans.head()

In [ ]:
df_data_en.to_csv('data_Stem.csv', index=False)

#### Menampilkan hasil *data preprocessing*

In [ ]:
data_clean = data_cleans.dropna()
data_clean.to_csv('data_clean.csv', index=False)
data_cleans = pd.read_csv('data_clean.csv', encoding='latin1')

In [ ]:
data_cleans = pd.read_csv('data_clean.csv', encoding='latin1')
data_cleans.head()

In [ ]:
data_cleans.sample(6, random_state = 134)

#### Mengecek data *missing value*

In [ ]:
data_cleans.info()

In [ ]:
len(data_cleans)

#### Melakukan pelabelan pada data ulasan aplikasi Peduli Lindungi

In [ ]:
sia = SentimentIntensityAnalyzer()
sia.polarity_scores("The film was awesome")

In [ ]:
sia.polarity_scores("I liked this music but it is not good as the other one")

In [ ]:
data_cleans["Ulasan"][0:10].apply(lambda x: sia.polarity_scores(x))

In [ ]:
data_cleans['Ulasan'][0:10].apply(lambda x: sia.polarity_scores(x)["compound"])

In [ ]:
data_cleans["polarity_score"] = data_cleans['Ulasan'].apply(lambda x: sia.polarity_scores(x)["compound"])
data_cleans.head()

In [ ]:
data_cleans["Ulasan"][0:10].apply(lambda x: "pos" if sia.polarity_scores(x)["compound"] > 0 else "neg")

In [ ]:
data_cleans["sentiment_label"] = data_cleans["Ulasan"].apply(lambda x: 1 if sia.polarity_scores(x)["compound"] > 0 else 0)
data_cleans.head()

In [ ]:
data_cleans["sentiment_label"].value_counts()

In [ ]:
data_cleans = data_cleans.astype({'sentiment_label': 'category'})
data_cleans = data_cleans.astype({'Ulasan': 'string'})

#### Melakukan pembobotan *TF-IDF*

In [ ]:
tf = TfidfVectorizer()
text_tf = tf.fit_transform(data_cleans['Ulasan'].astype('U'))
text_tf[0]

In [ ]:
tf_count = data_cleans['Ulasan'].apply(lambda x: pd.value_counts(x.split(" "))).sum(axis=0).reset_index()

tf_count.columns = ["words", "tf"]
tf_count.head()

In [ ]:
tf_count[(tf_count['words'] == "drain") | (tf_count['words'] == "batteri") | (tf_count['words'] == "good") | (tf_count['words'] == "better") | (tf_count['words'] == "last") | (tf_count['words'] == "year")]

#### Membagi Data (*data training* sebanyak 80 persen dan *data testing* sebanyak 20 persen)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(text_tf, data_cleans['sentiment_label'], test_size=0.2, random_state=42)
# x1_train, x1_test, y1_train, y1_test = train_test_split(text_tf, data_cleans['sentiment_label'], test_size=0.25, random_state=42)
# x2_train, x2_test, y2_train, y2_test = train_test_split(text_tf, data_cleans['sentiment_label'], test_size=0.3, random_state=42)

In [ ]:
x1_train, x1_test, y1_train, y1_test = train_test_split(data_cleans['Ulasan'], data_cleans['sentiment_label'], test_size=0.2, random_state=42)

In [ ]:
x1_test.sample(1)

In [ ]:
x1_test[5274], y1_test[5274]

In [ ]:
Sentimen_testing = y1_test.to_frame().reset_index(False)
testing_pos = Sentimen_testing[Sentimen_testing['sentiment_label']==1]
testing_neg = Sentimen_testing[Sentimen_testing['sentiment_label']==0]

In [ ]:
index_test_pos = []
index_test_neg = []
for i in testing_pos['index']:
  index_test_pos.append(i)
for i in testing_neg['index']:
  index_test_neg.append(i)

In [ ]:
test_pos = pd.DataFrame()
for i in index_test_pos:
  test_pos.append(data_cleans['Ulasan'].iloc[i])

test_pos

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

#### Proses ADASYN

In [ ]:
data_ada = data_cleans.sample(10, random_state = 46).reset_index().drop("index", axis=1)
data_ada

In [ ]:
data_ada['sentiment_label'].value_counts()

In [ ]:
tf = TfidfVectorizer()
text_tf_ada = tf.fit_transform(data_ada['Ulasan'].astype('U'))
text_tf_ada

In [ ]:
from scipy.spatial.distance import euclidean
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors

k = 5

# Build K Nearest Neighbors model
knn_model = NearestNeighbors(n_neighbors=k, metric="euclidean").fit(text_tf_ada)
distances, indices = knn_model.kneighbors()


# Print the 'k' nearest neighbors
print("K Nearest Neighbors dg pusat D1:")
for rank, index in enumerate(indices[1][:k], start=1):
    print(str(rank) + " ==>", index)
print("K Nearest Neighbors dg pusat D6")
for rank, index in enumerate(indices[6][:k], start=1):
    print(str(rank) + " ==>", index)
print("K Nearest Neighbors dg pusat D9")
for rank, index in enumerate(indices[9][:k], start=1):
    print(str(rank) + " ==>", index)    

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

a = [0, 8, 7, 4, 2]
b = [4, 8, 7, 0, 5]
c = [7, 8, 4, 2, 6]
print("Jarak euclidean K Nearest Neighbors dg pusat D0:")
for i in a:
  print(str(i), "==>", euclidean_distances(text_tf_ada[0],text_tf_ada[i]))
print("Jarak euclidean K Nearest Neighbors dg pusat D1:")
for i in b:
  print(str(i), "==>", euclidean_distances(text_tf_ada[6],text_tf_ada[i]))
print("Jarak euclidean K Nearest Neighbors dg pusat D2:")
for i in c:
  print(str(i), "==>", euclidean_distances(text_tf_ada[9],text_tf_ada[i]))

## Algoritma Multinomial Naive Bayes

#### Simulasi MNB

In [ ]:
# data training
data_cleans.sample(30, random_state =134).drop('polarity_score', axis = 1)

In [ ]:
# data testing
data_cleans.sample(1, random_state = 300).drop('polarity_score', axis = 1)

### Imbalanced Data Modelling





In [ ]:
clf = MultinomialNB().fit(x_train,y_train)
predicted = clf.predict(x_test)
false_positive_rate, true_positive_rate, tresholds = roc_curve(y_test,clf.predict(x_test))
print('MultinomialNB Accuracy: ', accuracy_score(y_test,predicted))
print('MultinomialNB Precision: ', precision_score(y_test,predicted, average='binary', pos_label=0))
print('MultinomialNB Recall: ', recall_score(y_test,predicted, average='binary', pos_label=0))
print('MultinomialNB f1-score: ', f1_score(y_test,predicted, average='binary', pos_label=0))
print('MultinomialNB AUC: ', auc(false_positive_rate, true_positive_rate))

print(f'confusion matrix:\n {confusion_matrix(y_test,predicted)}')
print('======================================\n')
print(classification_report(y_test,predicted,zero_division=0))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, auc, roc_curve

clf = MultinomialNB().fit(x1_train,y1_train)
predicted = clf.predict(x1_test)
false_positive_rate, true_positive_rate, tresholds = roc_curve(y1_test,clf.predict_proba(x1_test)[:,1])
print('MultinomialNB Accuracy: ', accuracy_score(y1_test,predicted))
print('MultinomialNB Precision: ', precision_score(y1_test,predicted, average='binary', pos_label=0))
print('MultinomialNB Recall: ', recall_score(y1_test,predicted, average='binary', pos_label=0))
print('MultinomialNB f1-score: ', f1_score(y1_test,predicted, average='binary', pos_label=0))
print('MultinomialNB AUC: ', auc(false_positive_rate, true_positive_rate))

print(f'confusion matrix:\n {confusion_matrix(y1_test,predicted)}')
print('======================================\n')
print(classification_report(y1_test,predicted,zero_division=0))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, auc, roc_curve

clf = MultinomialNB().fit(x2_train,y2_train)
predicted = clf.predict(x2_test)
false_positive_rate, true_positive_rate, tresholds = roc_curve(y2_test,clf.predict_proba(x2_test)[:,1])
print('MultinomialNB Accuracy: ', accuracy_score(y2_test,predicted))
print('MultinomialNB Precision: ', precision_score(y2_test,predicted, average='binary', pos_label=0))
print('MultinomialNB Recall: ', recall_score(y2_test,predicted, average='binary', pos_label=0))
print('MultinomialNB f1-score: ', f1_score(y2_test,predicted, average='binary', pos_label=0))
print('MultinomialNB AUC: ', auc(false_positive_rate, true_positive_rate))



print(f'confusion matrix:\n {confusion_matrix(y2_test,predicted)}')
print('======================================\n')
print(classification_report(y2_test,predicted,zero_division=0))

### SMOTE Modelling

In [ ]:
from imblearn.over_sampling import SMOTE

x = x_train
y = y_train

print(x_train.shape)
print(y_train.shape)
print(y_train.value_counts())

smote = SMOTE(sampling_strategy= 'minority')
x_sm, y_sm =smote.fit_resample(x_train, y_train)

print(y_sm.value_counts())
print(y_test.value_counts())

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, auc, roc_curve

clf = MultinomialNB().fit(x_sm,y_sm)
predicted = clf.predict(x_test)
false_positive_rate, true_positive_rate, tresholds = roc_curve(y_test, clf.predict(x_test))
print('MultinomialNB Accuracy: ', accuracy_score(y_test, predicted))
print('MultinomialNB Precision: ', precision_score(y_test,predicted, average='binary', pos_label=0))
print('MultinomialNB Recall: ', recall_score(y_test,predicted, average='binary', pos_label=0))
print('MultinomialNB f1-score: ', f1_score(y_test,predicted, average='binary', pos_label=0))
#print('MultinomialNB AUC: ', roc_auc_score(y_test,clf.predict_proba(x_test)[:,1]))
print('MultinomialNB AUC: ', auc(false_positive_rate, true_positive_rate))

print(f'confusion matrix:\n {confusion_matrix(y_test,predicted)}')
print('======================================\n')
print(classification_report(y_test,predicted,zero_division=0))

In [ ]:
#extracting true_positives, false_positives, true_negatives, false_negatives
tn, fp, fn, tp = confusion_matrix(y_test, predicted).ravel()
print("True Negatives: ",tn)
print("False Positives: ",fp)
print("False Negatives: ",fn)
print("True Positives: ",tp)

In [ ]:
import matplotlib.pyplot as plt
import numpy
from sklearn import metrics

confusion_matrix = metrics.confusion_matrix(y_test,predicted)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()
plt.show()

## ADASYN Modelling

In [ ]:
from imblearn.over_sampling import ADASYN

x = x_train
y = y_train

print(x.shape)
print(y.shape)
print(y.value_counts())

ada = ADASYN(sampling_strategy= 'minority')
x_ada, y_ada =ada.fit_resample(x, y)

print(y_ada.value_counts())

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score,auc,roc_curve

clf = MultinomialNB().fit(x_ada,y_ada)
predicted = clf.predict(x_test)
false_positive_rate, true_positive_rate, tresholds = roc_curve(y_test,clf.predict(x_test))
print('MultinomialNB Accuracy: ', accuracy_score(y_test,predicted))
print('MultinomialNB Precision: ', precision_score(y_test,predicted, average='binary', pos_label=0))
print('MultinomialNB Recall: ', recall_score(y_test,predicted, average='binary', pos_label=0))
print('MultinomialNB f1-score: ', f1_score(y_test,predicted, average='binary', pos_label=0))
print('MultinomialNB AUC: ', auc(false_positive_rate, true_positive_rate))

print(f'confusion matrix:\n {confusion_matrix(y_test,predicted)}')
print('======================================\n')
print(classification_report(y_test,predicted,zero_division=0))

In [ ]:
import matplotlib.pyplot as plt
import numpy
from sklearn import metrics

confusion_matrix = metrics.confusion_matrix(y_test,predicted)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()
plt.show()

In [ ]:
#extracting true_positives, false_positives, true_negatives, false_negatives
tn, fp, fn, tp = metrics.confusion_matrix(y_test, predicted).ravel()
print("True Negatives: ",tn)
print("False Positives: ",fp)
print("False Negatives: ",fn)
print("True Positives: ",tp)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data_cleans["Ulasan"], data_cleans['sentiment_label'], test_size=0.2, random_state=42)

In [ ]:
#Write  predictions to file
x_test = pd.DataFrame(x_test).reset_index().drop('index', axis = 1)
test_y = pd.DataFrame(y_test).reset_index().drop('index', axis = 1) #Actual values
yhat = pd.DataFrame(predicted) #predicted values using the model

# x_test.rename(columns= {0: "Test Data"}, inplace = True)
test_y.rename(columns= {0: 'Label Sentimen'}, inplace = True)
yhat.rename(columns = {0: 'Hasil Prediksi'}, inplace = True)

new = pd.concat([x_test, test_y, yhat], axis = 1)
#print(('ct'+ str(i)))
new.to_excel('Hasil_testing.xlsx', index = False)

In [ ]:
x_test.head()

## SMOTE + Tomek Modelling

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer()
text_tf5 = tf.fit_transform(data_cleans['content'].astype('U'))
text_tf5

In [ ]:
from imblearn.combine import SMOTETomek

x = text_tf5
y = data_cleans['sentiment_label']

print(x.shape)
print(y.shape)
print(y.value_counts())

smtom = SMOTETomek(sampling_strategy= 'minority')
x_smtom, y_smtom =smtom.fit_resample(x, y)

print(y_smtom.value_counts())

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

clf = MultinomialNB().fit(x_smtom,y_smtom.ravel())
predicted = clf.predict(x_test)
print('MultinomialNB Accuracy: ', accuracy_score(y_test,predicted))
print('MultinomialNB Precision: ', precision_score(y_test,predicted, average='binary', pos_label="neg"))
print('MultinomialNB Recall: ', recall_score(y_test,predicted, average='binary', pos_label="neg"))
print('MultinomialNB f1-score: ', f1_score(y_test,predicted, average='binary', pos_label="neg"))
print('MultinomialNB AUC: ', roc_auc_score(y_test,clf.predict_proba(x_test)[:,1]))

print(f'confusion matrix:\n {confusion_matrix(y_test,predicted)}')
print('======================================\n')
print(classification_report(y_test,predicted,zero_division=0))

## SMOTE + ENN Modelling

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer()
text_tf6 = tf.fit_transform(data_cleans['content'].astype('U'))
text_tf6

In [ ]:
from imblearn.combine import SMOTEENN

x = text_tf6
y = data_cleans['sentiment_label']

print(x.shape)
print(y.shape)
print(y.value_counts())

smenn = SMOTEENN(sampling_strategy= 'minority')
x_smenn, y_smenn =smenn.fit_resample(x, y)

print(y_smenn.value_counts())

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

clf = MultinomialNB().fit(x_smenn,y_smenn.ravel())
predicted = clf.predict(x_test)
print('MultinomialNB Accuracy: ', accuracy_score(y_test,predicted))
print('MultinomialNB Precision: ', precision_score(y_test,predicted, average='binary', pos_label="neg"))
print('MultinomialNB Recall: ', recall_score(y_test,predicted, average='binary', pos_label="neg"))
print('MultinomialNB f1-score: ', f1_score(y_test,predicted, average='binary', pos_label="neg"))
print('MultinomialNB AUC: ', roc_auc_score(y_test,clf.predict_proba(x_test)[:,1]))

print(f'confusion matrix:\n {confusion_matrix(y_test,predicted)}')
print('======================================\n')
print(classification_report(y_test,predicted,zero_division=0))

## Visualisasi 

In [ ]:
!pip install wordcloud

In [ ]:
data_cleans['sentiment_label'].value_counts()

In [ ]:
data_cleans.info()

In [ ]:
from textblob import Word, TextBlob
from wordcloud import WordCloud,STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
sentimen_pos = data_cleans[data_cleans['sentiment_label']==1]
sentimen_neg = data_cleans[data_cleans['sentiment_label']==0]
sentimen_pos.to_csv('sentimen_positif.csv', index=False)
sentimen_neg.to_csv('sentimen_negatif.csv', index=False)

In [ ]:
import pandas as pd
visual_pos = pd.read_csv('sentimen_positif.csv', encoding='latin1')
visual_pos.head()

In [ ]:
visual_neg = pd.read_csv('sentimen_negatif.csv', encoding='latin1')
visual_neg.head()

In [ ]:
tf_pos = visual_pos['Ulasan'].apply(lambda x: pd.value_counts(x.split(" "))).sum(axis=0).reset_index()

tf_pos.columns = ["words", "tf"]
tf_pos.head()

In [ ]:
tf_neg = visual_neg['Ulasan'].apply(lambda x: pd.value_counts(x.split(" "))).sum(axis=0).reset_index()

tf_neg.columns = ["words", "tf"]
tf_neg.head()

In [ ]:
tf_pos[tf_pos["words"]=="good"]

In [ ]:
tf_pos.shape, tf_neg.shape

In [ ]:
tf_pos["words"].nunique(), tf_neg["words"].nunique()

In [ ]:
tf_pos["tf"].describe([0.05, 0.10, 0.25, 0.50, 0.75, 0.80, 0.90, 0.95, 0.99]).T

In [ ]:
tf_neg["tf"].describe([0.05, 0.10, 0.25, 0.50, 0.75, 0.80, 0.90, 0.95, 0.99]).T

In [ ]:
# Barplot sentimen positif

tf_pos[tf_pos["tf"] > 200].plot.bar(x="words", y="tf")
plt.show()

In [ ]:
# Barplot sentimen negatif

tf_neg[tf_neg["tf"] > 310].plot.bar(x="words", y="tf")
plt.show()

In [ ]:
# Wordcloud sentimen positif

text = " ".join(i for i in visual_pos['Ulasan'])
wordcloud = WordCloud(max_font_size=50,
                      max_words=200,
                      background_color="black",width=250, height=250).generate(text)
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.savefig('sentimen positif.png', format='png')
plt.show()

In [ ]:
# Wordcloud sentimen negatif

text = " ".join(i for i in visual_neg['Ulasan'])
wordcloud = WordCloud(max_font_size=50,
                      max_words=200,
                      background_color="black",width=250, height=250).generate(text)
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.savefig('sentimen negatif.png', format='png')
plt.show()